In [4]:
from typing import List
from itertools import combinations

# BCH(15,7,2) の生成多項式: g(x) = x⁸ + x⁷ + x⁶ + x⁴ + x² + 1
G = [1,1,1,0,1,0,1,0,1]

# GF(2) 多項式加算（XOR）
def poly_add(a: List[int], b: List[int]) -> List[int]:
    length = max(len(a), len(b))
    a = [0]*(length - len(a)) + a
    b = [0]*(length - len(b)) + b
    return [(ai ^ bi) for ai, bi in zip(a, b)]

# mod2 除算：戻り値は余り
def poly_mod(dividend: List[int], divisor: List[int]) -> List[int]:
    dividend = dividend[:]
    while len(dividend) >= len(divisor):
        if dividend[0] == 1:
            for i in range(len(divisor)):
                dividend[i] ^= divisor[i]
        dividend.pop(0)
    return dividend

# BCH(15,7,2) エンコード
def bch15_7_2_encode(data_bits: List[int]) -> List[int]:
    if len(data_bits) != 7:
        raise ValueError("7ビットのデータが必要です")
    dividend = data_bits + [0]*8
    remainder = poly_mod(dividend, G)
    return data_bits + remainder

# シンドローム（誤り検出の指標）
def compute_syndrome(codeword: List[int]) -> List[int]:
    return poly_mod(codeword, G)

# BCH(15,7,2) デコード（最大2ビット訂正）
def bch15_7_2_decode(codeword: List[int]) -> (List[int], int):
    original_syndrome = compute_syndrome(codeword)
    if all(x == 0 for x in original_syndrome):
        return codeword[:7], 0  # 誤りなし

    # 1ビット誤り探索
    for i in range(len(codeword)):
        trial = codeword[:]
        trial[i] ^= 1
        if all(x == 0 for x in compute_syndrome(trial)):
            return trial[:7], 1

    # 2ビット誤り探索
    for i, j in combinations(range(len(codeword)), 2):
        trial = codeword[:]
        trial[i] ^= 1
        trial[j] ^= 1
        if all(x == 0 for x in compute_syndrome(trial)):
            return trial[:7], 2

    return codeword[:7], -1  # 訂正不能

# バイナリ表示
def to_bitstring(bits: List[int]) -> str:
    return ''.join(str(b) for b in bits)

# === 実行例 ===

# 入力データ（7ビット）
data = [1, 0, 1, 1, 0, 1, 1]

# エンコード
codeword = bch15_7_2_encode(data)

# 誤り挿入（例：ビット3と5を反転）
codeword_err = codeword[:]
codeword_err[3] ^= 1
codeword_err[5] ^= 1

# デコードと訂正
decoded, corrected = bch15_7_2_decode(codeword_err)

# 表示
print("=== BCH(15,7,2) Demo ===")
print("Original Data : ", to_bitstring(data))
print("Codeword      : ", to_bitstring(codeword))
print("Corrupted     : ", to_bitstring(codeword_err))
print("Decoded       : ", to_bitstring(decoded))
print("Corrected Bits: ", corrected)


=== BCH(15,7,2) Demo ===
Original Data :  1011011
Codeword      :  101101101001100
Corrupted     :  101000101001100
Decoded       :  1000001
Corrected Bits:  2
